In [2]:
#Build a DNN with 5 hidden layers of 100 neurals each, He initialization and ELU activation
import tensorflow as tf 
import numpy as np

n_input=28*28
n_output=5#Only 0-4 for MNIST
n_neural=100#number of neurals in each hidden layer


def hidden_layer(X, n_output, activation='None'):
    n_input=int(X.get_shape()[1])
    #stdd=2.0*np.sqrt(1/(n_input+n_output))
    stdd=2/np.sqrt(n_input)
    init=tf.truncated_normal((n_input, n_output), stddev=stdd)
    W=tf.Variable(init, name='Weights')
    b=tf.Variable(tf.zeros([n_output]), name='Bias')
    z=tf.matmul(X, W)+b
    if activation=='ELU':
        return tf.nn.elu(z)
    elif activation=='softmax':
        return tf.nn.softmax(z)
    else:
        return z

X=tf.placeholder(tf.float32, shape=(None, n_input), name='Images')
y=tf.placeholder(tf.int64, shape=(None), name='Labels')

hidden_1=hidden_layer(X, n_neural, activation='ELU')
hidden_2=hidden_layer(hidden_1, n_neural, activation='ELU')
hidden_3=hidden_layer(hidden_2, n_neural, activation='ELU')
hidden_4=hidden_layer(hidden_3, n_neural, activation='ELU')
logits=hidden_layer(hidden_4, n_output, activation='softmax')

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/")

X_training, y_training=mnist.train.images, mnist.train.labels
X_testing, y_testing=mnist.test.images, mnist.test.labels

def extract(X, y, target):
    X_target=[]
    y_target=[]
    for i in range(X.shape[0]):
        if(y[i] in target):
            X_target.append(X[i])
            y_target.append(y[i])
    return np.array(X_target), np.array(y_target)



# X_train1 = mnist.train.images[mnist.train.labels < 5]
# y_train1 = mnist.train.labels[mnist.train.labels < 5]
# X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
# y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
# X_test1 = mnist.test.images[mnist.test.labels < 5]
# y_test1 = mnist.test.labels[mnist.test.labels < 5]


target=[0,1,2,3,4]
X_training_04, y_training_04=extract(X_training, y_training, target)
X_testing_04, y_testing_04=extract(X_testing, y_testing, target)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
X_training_04.shape

(28038, 784)

In [5]:
xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss=tf.reduce_mean(xentropy, name='loss')

learning_rate=0.01
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(loss)

correct=tf.nn.in_top_k(logits, y, 1)
accuracy=tf.reduce_mean(tf.cast(correct, tf.float32))

In [6]:
print(X_testing_04.shape)
print(y_testing_04.shape)

(5139, 784)
(5139,)


In [7]:
import time
n_epochs=1000
init=tf.global_variables_initializer()
saver = tf.train.Saver()

start=time.time()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X:X_training_04, y:y_training_04})
        training_accuracy=accuracy.eval(feed_dict={X:X_training_04, y:y_training_04})
        testing_accuracy=accuracy.eval(feed_dict={X:X_testing_04, y:y_testing_04})
        print("Epoch: ", epoch, "training: ", training_accuracy, "testing: ", testing_accuracy)
    save_path = saver.save(sess, "/home/emmittxu/Desktop/try_tensorflow/models/mnist_model.ckpt")
    print("Model saved in file: %s" % save_path)
end=time.time()
print("Completed in ", end-start)

('Epoch: ', 0, 'training: ', 0.40216848, 'testing: ', 0.3987157)
('Epoch: ', 1, 'training: ', 0.59840214, 'testing: ', 0.60556531)
('Epoch: ', 2, 'training: ', 0.56059635, 'testing: ', 0.56606346)
('Epoch: ', 3, 'training: ', 0.53541625, 'testing: ', 0.54913408)
('Epoch: ', 4, 'training: ', 0.54811329, 'testing: ', 0.55536097)
('Epoch: ', 5, 'training: ', 0.55581713, 'testing: ', 0.56684178)
('Epoch: ', 6, 'training: ', 0.57397103, 'testing: ', 0.58396578)
('Epoch: ', 7, 'training: ', 0.58363652, 'testing: ', 0.59447366)
('Epoch: ', 8, 'training: ', 0.58452815, 'testing: ', 0.59661412)
('Epoch: ', 9, 'training: ', 0.5838505, 'testing: ', 0.59447366)
('Epoch: ', 10, 'training: ', 0.58299452, 'testing: ', 0.59350067)
('Epoch: ', 11, 'training: ', 0.58420718, 'testing: ', 0.59505743)
('Epoch: ', 12, 'training: ', 0.58638275, 'testing: ', 0.5973925)
('Epoch: ', 13, 'training: ', 0.58866537, 'testing: ', 0.59953296)
('Epoch: ', 14, 'training: ', 0.58966404, 'testing: ', 0.60031134)
('Epoch:

In [3]:
#Introduce batch normalization
from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.layers import fully_connected
import tensorflow as tf 
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

n_input=28*28
n_output=5#Only 0-4 for MNIST
n_neural=100#number of neurals in each hidden layer


mnist=input_data.read_data_sets("/tmp/data/")

X_training, y_training=mnist.train.images, mnist.train.labels
X_testing, y_testing=mnist.test.images, mnist.test.labels

def extract(X, y, target):
    X_target=[]
    y_target=[]
    for i in range(X.shape[0]):
        if(y[i] in target):
            X_target.append(X[i])
            y_target.append(y[i])
    return np.array(X_target), np.array(y_target)
target=[0,1,2,3,4]
X_training_04, y_training_04=extract(X_training, y_training, target)
X_testing_04, y_testing_04=extract(X_testing, y_testing, target)


def hidden_layer(X, n_output, activation='None'):
    n_input=int(X.get_shape()[1])
    #stdd=2.0*np.sqrt(1/(n_input+n_output))
    stdd=2/np.sqrt(n_input)
    init=tf.truncated_normal((n_input, n_output), stddev=stdd)
    W=tf.Variable(init, name='Weights')
    b=tf.Variable(tf.zeros([n_output]), name='Bias')
    z=tf.matmul(X, W)+b
    if activation=='ELU':
        return tf.nn.elu(z)
    elif activation=='softmax':
        return tf.nn.softmax(z)
    else:
        return z

X=tf.placeholder(tf.float32, shape=(None, n_input), name='Images')
y=tf.placeholder(tf.int64, shape=(None), name='Labels')

is_training=tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params={
    'is_training':is_training,
    'decay':0.99,
    'updates_collections':None
}

hidden1=fully_connected(X, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden2=fully_connected(hidden1, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden3=fully_connected(hidden2, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden4=fully_connected(hidden3, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
logits=fully_connected(hidden4, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)

xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss=tf.reduce_mean(xentropy, name='loss')

learning_rate=0.01
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(loss)

correct=tf.nn.in_top_k(logits, y, 1)
accuracy=tf.reduce_mean(tf.cast(correct, tf.float32))

n_epochs=1000
init=tf.global_variables_initializer()
saver = tf.train.Saver()

start=time.time()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={is_training:True, X:X_training_04, y:y_training_04})
        training_accuracy=accuracy.eval(feed_dict={is_training:True, X:X_training_04, y:y_training_04})
        testing_accuracy=accuracy.eval(feed_dict={is_training:False, X:X_testing_04, y:y_testing_04})
        print("Epoch: ", epoch, "training: ", training_accuracy, "testing: ", testing_accuracy)
    save_path = saver.save(sess, "/home/emmittxu/Desktop/try_tensorflow/models/mnist_bn_model.ckpt")
    print("Model saved in file: %s" % save_path)
end=time.time()
print("Completed in ", end-start)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
('Epoch: ', 0, 'training: ', 0.38305157, 'testing: ', 0.20198482)
('Epoch: ', 1, 'training: ', 0.50381625, 'testing: ', 0.20081729)
('Epoch: ', 2, 'training: ', 0.55720806, 'testing: ', 0.20081729)
('Epoch: ', 3, 'training: ', 0.60350239, 'testing: ', 0.20081729)
('Epoch: ', 4, 'training: ', 0.6481204, 'testing: ', 0.20081729)
('Epoch: ', 5, 'training: ', 0.667059, 'testing: ', 0.20081729)
('Epoch: ', 6, 'training: ', 0.67911404, 'testing: ', 0.20081729)
('Epoch: ', 7, 'training: ', 0.68710321, 'testing: ', 0.20081729)
('Epoch: ', 8, 'training: ', 0.69238174, 'testing: ', 0.20081729)
('Epoch: ', 9, 'training: ', 0.70037091, 'testing: ', 0.20081729)
('Epoch: ', 10, 'training: ', 0.71499395, 'testing: ', 0.20081729)
('Epoch: ', 11, 'training: ', 0.73378986, 'testing: ', 0.20081729)
('Epoch: ', 12, 'tr

In [10]:
#Introduce batch normalization with mini batch and early stoping
from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.layers import fully_connected
import tensorflow as tf 
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

n_input=28*28
n_output=5#Only 0-4 for MNIST
n_neural=140#number of neurals in each hidden layer


mnist=input_data.read_data_sets("/tmp/data/")

# X_training, y_training=mnist.train.images, mnist.train.labels
# X_testing, y_testing=mnist.test.images, mnist.test.labels

X_training_04 = mnist.train.images[mnist.train.labels < 5]
y_training_04 = mnist.train.labels[mnist.train.labels < 5]
X_valid_04 = mnist.validation.images[mnist.validation.labels < 5]
y_valid_04 = mnist.validation.labels[mnist.validation.labels < 5]
X_testing_04 = mnist.test.images[mnist.test.labels < 5]
y_testing_04 = mnist.test.labels[mnist.test.labels < 5]

# def extract(X, y, target):
#     X_target=[]
#     y_target=[]
#     for i in range(X.shape[0]):
#         if(y[i] in target):
#             X_target.append(X[i])
#             y_target.append(y[i])
#     return np.array(X_target), np.array(y_target)
# target=[0,1,2,3,4]
# X_training_04, y_training_04=extract(X_training, y_training, target)
# X_testing_04, y_testing_04=extract(X_testing, y_testing, target)


def hidden_layer(X, n_output, activation='None'):
    n_input=int(X.get_shape()[1])
    #stdd=2.0*np.sqrt(1/(n_input+n_output))
    stdd=2/np.sqrt(n_input)
    init=tf.truncated_normal((n_input, n_output), stddev=stdd)
    W=tf.Variable(init, name='Weights')
    b=tf.Variable(tf.zeros([n_output]), name='Bias')
    z=tf.matmul(X, W)+b
    if activation=='ELU':
        return tf.nn.elu(z)
    elif activation=='softmax':
        return tf.nn.softmax(z)
    else:
        return z

X=tf.placeholder(tf.float32, shape=(None, n_input), name='Images')
y=tf.placeholder(tf.int64, shape=(None), name='Labels')

is_training=tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params={
    'is_training':is_training,
    'decay':0.99,
    'updates_collections':None
}

hidden1=fully_connected(X, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden2=fully_connected(hidden1, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden3=fully_connected(hidden2, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden4=fully_connected(hidden3, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)
logits=fully_connected(hidden4, n_output, normalizer_fn=batch_norm, normalizer_params=bn_params)

xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss=tf.reduce_mean(xentropy, name='loss')

learning_rate=0.01

optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(loss)

correct=tf.nn.in_top_k(logits, y, 1)
accuracy=tf.reduce_mean(tf.cast(correct, tf.float32))

n_epochs=1000
batch_size=500
n_batches=X_training_04.shape[0]//batch_size
best_validation_loss=np.infty
max_checks_no_progress=20
check_without_process=0
init=tf.global_variables_initializer()
saver = tf.train.Saver()

start=time.time()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch in range(n_batches):
            rnd_index=np.random.permutation(X_training_04.shape[0])
            X_batch=X_training_04[batch_size*batch:batch_size*(batch+1)]
            y_batch=y_training_04[batch_size*batch:batch_size*(batch+1)]
            sess.run(training_op, feed_dict={is_training:True, X:X_batch, y:y_batch})
        training_accuracy=accuracy.eval(feed_dict={is_training:True, X:X_training_04, y:y_training_04})
        validation_accuracy=accuracy.eval(feed_dict={is_training:False, X:X_valid_04, y:y_valid_04})
        validation_loss=loss.eval(feed_dict={is_training:False, X:X_valid_04, y:y_valid_04})
        print("Epoch:", epoch, "training:", training_accuracy, "validation:", validation_accuracy, \
              "valid_loss", validation_loss, "best_loss", best_validation_loss)
        if(validation_loss<best_validation_loss):
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_validation_loss=validation_loss
            check_without_process=0
        else:
            check_without_process+=1
            if check_without_process>=max_checks_no_progress:
                print("Early stopping!")
                break;
    testing_accuracy=accuracy.eval(feed_dict={is_training:False, X:X_testing_04, y:y_testing_04})
    print("Final model on testing: ", testing_accuracy)
end=time.time()
print("Completed in ", end-start)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
('Epoch:', 0, 'training:', 0.94717884, 'validation:', 0.66614544, 'valid_loss', 1.2976019, 'best_loss', inf)
('Epoch:', 1, 'training:', 0.96540409, 'validation:', 0.70914775, 'valid_loss', 1.0809054, 'best_loss', 1.2976019)
('Epoch:', 2, 'training:', 0.9723233, 'validation:', 0.91282254, 'valid_loss', 0.84137475, 'best_loss', 1.0809054)
('Epoch:', 3, 'training:', 0.97549754, 'validation:', 0.96247065, 'valid_loss', 0.6342715, 'best_loss', 0.84137475)
('Epoch:', 4, 'training:', 0.97838646, 'validation:', 0.96989834, 'valid_loss', 0.49021199, 'best_loss', 0.6342715)
('Epoch:', 5, 'training:', 0.98009843, 'validation:', 0.97185302, 'valid_loss', 0.4210445, 'best_loss', 0.49021199)
('Epoch:', 6, 'training:', 0.98091877, 'validation:', 0.97068024, 'valid_loss', 0.37992233, 'best_loss', 0.4210445)
('Epoch

In [1]:
#####################################################
#Define a customized DNN class that can be used in sklearn random search to fine tune parameters
#####################################################
from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.layers import fully_connected
import tensorflow as tf 
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
from sklearn.base import BaseEstimator, TransformerMixin
he_init = tf.contrib.layers.variance_scaling_initializer()
class My_DNN(BaseEstimator, TransformerMixin):
    def __init__(self, X_testing=None, y_testing=None, X_validation=None, y_validation=None, n_input=28*28, n_output=5, n_neurons=140, n_layers=5, learning_rate=0.01, n_epoches=1000,\
                 batch_size=100,batch_norm_momentum=None, dropout_rate=None, activation=tf.nn.elu, \
                 initializer=he_init):
#         self.X_training=X_training
#         self.y_training=y_training
        self.X_testing=X_testing
        self.y_testing=y_testing
        self.X_validation=X_validation
        self.y_validation=y_validation
        self.n_input=n_input
        self.n_output=n_output
        self.n_neurons=n_neurons
        self.n_layers=n_layers
        self.learning_rate=learning_rate
        self.n_epoches=n_epoches
        self.batch_size=batch_size
        self.batch_norm_momentum=batch_norm_momentum
        self.dropout_rate=dropout_rate
        self.activation=activation
        self.initializer=initializer
        self._session = None
    
    def build_hidden_layers(self, input):
        is_training=tf.placeholder_with_default(False, shape=(), name='training')
        for layer in range(self.n_layers-1):
            if(self.dropout_rate!=None):
                input=tf.layers.dropout(input, self.dropout_rate, training=is_training)
            input=tf.layers.dense(input, self.n_neurons, activation=self.activation,\
                                          kernel_initializer=self.initializer)
            if(self.batch_norm_momentum!=None):
                input=tf.layers.batch_normalization(input, momentum=self.batch_norm_momentum, training=is_training)
        self.is_training=is_training
        return input
    
    def build_graph(self):
        X=tf.placeholder(tf.float32, shape=(None, self.n_input), name='Images')
        y=tf.placeholder(tf.int64, shape=(None), name='Labels')       
        output=self.build_hidden_layers(X)
        logits=tf.layers.dense(output, self.n_output, activation=tf.nn.softmax, kernel_initializer=self.initializer)
        xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss=tf.reduce_mean(xentropy, name='loss')
        optimizer=tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        training_op=optimizer.minimize(loss)
        correct=tf.nn.in_top_k(logits, y, 1)
        accuracy=tf.reduce_mean(tf.cast(correct, tf.float32))
        Y_prob = tf.nn.softmax(logits, name="Y_proba")
        init=tf.global_variables_initializer()
        saver = tf.train.Saver()
        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._loss, self.Y_prob = loss, Y_prob
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver
        
    def fit(self, X_training, y_training):
        n_batches=X_training.shape[0]//self.batch_size
        best_validation_loss=np.infty
        max_checks_no_progress=20
        check_without_process=0
        
        start=time.time()
        self.build_graph()
        #with tf.Session() as sess:
        self._session = tf.Session()
        with self._session.as_default() as sess:
        #sess.run(self._init)
            self._init.run()
            for epoch in range(self.n_epoches):
                for batch in range(n_batches):
                    rnd_index=np.random.permutation(X_training.shape[0])
                    X_batch=X_training[self.batch_size*batch:self.batch_size*(batch+1)]
                    y_batch=y_training[self.batch_size*batch:self.batch_size*(batch+1)]
                    sess.run(self._training_op, feed_dict={self.is_training:True, self._X:X_batch, self._y:y_batch})
                training_accuracy=self._accuracy.eval(feed_dict={self.is_training:True, self._X:X_training, self._y:y_training})
                validation_accuracy=self._accuracy.eval(feed_dict={self.is_training:False, self._X:self.X_validation, self._y:self.y_validation})
                validation_loss=self._loss.eval(feed_dict={self.is_training:False, self._X:self.X_validation, self._y:self.y_validation})
                print("Epoch:", epoch, "training:", training_accuracy, "validation:", validation_accuracy, \
                      "valid_loss", validation_loss, "best_loss", best_validation_loss)
                if(validation_loss<best_validation_loss):
                    save_path = self._saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
                    best_validation_loss=validation_loss
                    check_without_process=0
                else:
                    check_without_process+=1
                    if check_without_process>=max_checks_no_progress:
                        print("Early stopping!")
                        break;
            testing_accuracy=self._accuracy.eval(feed_dict={self.is_training:False, self._X:self.X_testing, self._y:self.y_testing})
            print("Final model on testing: ", testing_accuracy)
        end=time.time()
        print("Completed in ", end-start)
        
    def predict_proba(self, X):
#         if not self._session:
#             raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        #with tf.Session() as sess:
        with self._session.as_default() as sess:
        #self._init.run()
            return self.Y_prob.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array(class_indices)

In [2]:
mnist=input_data.read_data_sets("/tmp/data/")

# X_training, y_training=mnist.train.images, mnist.train.labels
# X_testing, y_testing=mnist.test.images, mnist.test.labels

X_training_04 = mnist.train.images[mnist.train.labels < 5]
y_training_04 = mnist.train.labels[mnist.train.labels < 5]
X_valid_04 = mnist.validation.images[mnist.validation.labels < 5]
y_valid_04 = mnist.validation.labels[mnist.validation.labels < 5]
X_testing_04 = mnist.test.images[mnist.test.labels < 5]
y_testing_04 = mnist.test.labels[mnist.test.labels < 5]

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
my_dnn=My_DNN(X_testing=X_testing_04, y_testing=y_testing_04, X_validation=X_valid_04,\
              y_validation=y_valid_04, batch_size=500, batch_norm_momentum=0.99, dropout_rate=0.5)
my_dnn.fit(X_training_04, y_training_04)

('Epoch:', 0, 'training:', 0.93615806, 'validation:', 0.94370604, 'valid_loss', 0.96121371, 'best_loss', inf)
('Epoch:', 1, 'training:', 0.94311291, 'validation:', 0.95934325, 'valid_loss', 0.94554025, 'best_loss', 0.96121371)
('Epoch:', 2, 'training:', 0.94931877, 'validation:', 0.94761533, 'valid_loss', 0.95702839, 'best_loss', 0.94554025)
('Epoch:', 3, 'training:', 0.94714314, 'validation:', 0.93901485, 'valid_loss', 0.96584129, 'best_loss', 0.94554025)
('Epoch:', 4, 'training:', 0.95691562, 'validation:', 0.97224396, 'valid_loss', 0.93259579, 'best_loss', 0.94554025)
('Epoch:', 5, 'training:', 0.95670164, 'validation:', 0.97341675, 'valid_loss', 0.93142283, 'best_loss', 0.93259579)
('Epoch:', 6, 'training:', 0.95563161, 'validation:', 0.97263485, 'valid_loss', 0.9322049, 'best_loss', 0.93142283)
('Epoch:', 7, 'training:', 0.96041089, 'validation:', 0.97380763, 'valid_loss', 0.93103158, 'best_loss', 0.93142283)
('Epoch:', 8, 'training:', 0.95976889, 'validation:', 0.97146207, 'valid

In [5]:
predictions=my_dnn.predict(X_testing_04)
for i, j in zip(y_testing_04, predictions):
    print("label: ", i, "prediction ", j)

('label: ', 2, 'prediction ', 2)
('label: ', 1, 'prediction ', 1)
('label: ', 0, 'prediction ', 0)
('label: ', 4, 'prediction ', 4)
('label: ', 1, 'prediction ', 1)
('label: ', 4, 'prediction ', 4)
('label: ', 0, 'prediction ', 0)
('label: ', 0, 'prediction ', 0)
('label: ', 1, 'prediction ', 1)
('label: ', 3, 'prediction ', 3)
('label: ', 4, 'prediction ', 4)
('label: ', 4, 'prediction ', 4)
('label: ', 0, 'prediction ', 0)
('label: ', 4, 'prediction ', 4)
('label: ', 0, 'prediction ', 0)
('label: ', 1, 'prediction ', 1)
('label: ', 3, 'prediction ', 3)
('label: ', 1, 'prediction ', 1)
('label: ', 3, 'prediction ', 3)
('label: ', 4, 'prediction ', 4)
('label: ', 2, 'prediction ', 2)
('label: ', 1, 'prediction ', 1)
('label: ', 2, 'prediction ', 2)
('label: ', 1, 'prediction ', 1)
('label: ', 1, 'prediction ', 1)
('label: ', 4, 'prediction ', 4)
('label: ', 2, 'prediction ', 2)
('label: ', 3, 'prediction ', 3)
('label: ', 1, 'prediction ', 1)
('label: ', 2, 'prediction ', 2)
('label: '

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import time
def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [50,100, 150],
    "batch_size": [200,100, 500],
    "learning_rate": [0.01, 0.005, 0.02],
    "activation": [tf.nn.relu, tf.nn.elu],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    "n_layers": [3,5,7],
    "batch_norm_momentum": [0.99, None],
    "dropout_rate":[0.4, 0.5, 0.6, None],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}
start=time.time()
dnn_clf=My_DNN(X_testing_04[:3000], y_testing_04[:3000], X_valid_04[:1000], y_valid_04[:1000])
rnd_search = RandomizedSearchCV(dnn_clf, param_distributions = param_distribs,\
                                 scoring='accuracy', n_iter=50, random_state=42, verbose=2)
#rnd_search.fit(X_training_04, y_training_04, X_testing_04, y_testing_04, X_valid_04, y_valid_04)
#rnd_search.fit(X_training_04, y_training_04, X_testing_04, y_testing_04, X_valid_04, y_valid_04)
rnd_search.fit(X_training_04[:6000], y_training_04[:6000])
print("Complete in ", time.time()-start)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] dropout_rate=0.6, batch_norm_momentum=None, n_neurons=100, n_layers=5, batch_size=500, activation=<function relu at 0x7f9afeea0500>, learning_rate=0.01 
('Epoch:', 0, 'training:', 0.33225, 'validation:', 0.59100002, 'valid_loss', 1.326118, 'best_loss', inf)
('Epoch:', 1, 'training:', 0.44999999, 'validation:', 0.50199997, 'valid_loss', 1.3960782, 'best_loss', 1.326118)
('Epoch:', 2, 'training:', 0.54900002, 'validation:', 0.69099998, 'valid_loss', 1.2075752, 'best_loss', 1.326118)
('Epoch:', 3, 'training:', 0.63050002, 'validation:', 0.71600002, 'valid_loss', 1.1870078, 'best_loss', 1.2075752)
('Epoch:', 4, 'training:', 0.66724998, 'validation:', 0.72899997, 'valid_loss', 1.1752619, 'best_loss', 1.1870078)
('Epoch:', 5, 'training:', 0.67400002, 'validation:', 0.73500001, 'valid_loss', 1.168808, 'best_loss', 1.1752619)
('Epoch:', 6, 'training:', 0.69375002, 'validation:', 0.73199999, 'valid_loss', 1.1715631, 'best_loss',

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.8s remaining:    0.0s


('Epoch:', 0, 'training:', 0.36000001, 'validation:', 0.51499999, 'valid_loss', 1.3449793, 'best_loss', inf)
('Epoch:', 1, 'training:', 0.48199999, 'validation:', 0.63800001, 'valid_loss', 1.2635891, 'best_loss', 1.3449793)
('Epoch:', 2, 'training:', 0.57674998, 'validation:', 0.75999999, 'valid_loss', 1.1350434, 'best_loss', 1.2635891)
('Epoch:', 3, 'training:', 0.6415, 'validation:', 0.89399999, 'valid_loss', 1.0135021, 'best_loss', 1.1350434)
('Epoch:', 4, 'training:', 0.71625, 'validation:', 0.90799999, 'valid_loss', 0.99350059, 'best_loss', 1.0135021)
('Epoch:', 5, 'training:', 0.76924998, 'validation:', 0.92699999, 'valid_loss', 0.9775514, 'best_loss', 0.99350059)
('Epoch:', 6, 'training:', 0.81325001, 'validation:', 0.93300003, 'valid_loss', 0.97045726, 'best_loss', 0.9775514)
('Epoch:', 7, 'training:', 0.83200002, 'validation:', 0.935, 'valid_loss', 0.96910441, 'best_loss', 0.97045726)
('Epoch:', 8, 'training:', 0.83899999, 'validation:', 0.93900001, 'valid_loss', 0.96602613, '

In [11]:
rnd_search.best_params_

{'activation': <function tensorflow.python.ops.gen_nn_ops.relu>,
 'batch_norm_momentum': 0.99,
 'batch_size': 500,
 'dropout_rate': 0.5,
 'learning_rate': 0.01,
 'n_layers': 2,
 'n_neurons': 50}

In [12]:
rnd_search.best_score_

0.98056209430059205

In [13]:
my_dnn=My_DNN(X_testing=X_testing_04, y_testing=y_testing_04, X_validation=X_valid_04,\
              y_validation=y_valid_04, batch_size=500, batch_norm_momentum=0.99, dropout_rate=0.5, n_layers=2, n_neurons=50)
my_dnn.fit(X_training_04, y_training_04)

('Epoch:', 0, 'training:', 0.95342034, 'validation:', 0.92103207, 'valid_loss', 0.98697788, 'best_loss', inf)
('Epoch:', 1, 'training:', 0.96083885, 'validation:', 0.94018763, 'valid_loss', 0.96988183, 'best_loss', 0.98697788)
('Epoch:', 2, 'training:', 0.96643841, 'validation:', 0.92415953, 'valid_loss', 0.98065966, 'best_loss', 0.96988183)
('Epoch:', 3, 'training:', 0.97000498, 'validation:', 0.93706018, 'valid_loss', 0.97013527, 'best_loss', 0.96988183)
('Epoch:', 4, 'training:', 0.97382122, 'validation:', 0.95308834, 'valid_loss', 0.95387906, 'best_loss', 0.96988183)
('Epoch:', 5, 'training:', 0.97663885, 'validation:', 0.94331509, 'valid_loss', 0.9641245, 'best_loss', 0.95387906)
('Epoch:', 6, 'training:', 0.97881448, 'validation:', 0.96364349, 'valid_loss', 0.9401592, 'best_loss', 0.95387906)
('Epoch:', 7, 'training:', 0.97988445, 'validation:', 0.96207976, 'valid_loss', 0.9420653, 'best_loss', 0.9401592)
('Epoch:', 8, 'training:', 0.98027676, 'validation:', 0.96872556, 'valid_lo